In [1]:
!pip install ffmpeg-python

In [1]:
import ffmpeg
import numpy as np
import json
import tempfile
import os
from scipy.io import wavfile

In [2]:
# prompt: write a code to unzip саманчынын жолу-20250428T103309Z-001.zip file

# import zipfile

# with zipfile.ZipFile('саманчынын жолу-20250428T103309Z-001.zip', 'r') as zip_ref:
#     zip_ref.extractall()


In [3]:
audio_file = '/content/Aitmatov_turnalar.mp3'


In [4]:
# def generate_peaks_3(audio_file, output_json, samples=1000):
#     # Извлечение аудио
#     out, _ = ffmpeg.input(audio_file).output('pipe:1', format='s16le', ac=1, ar=44100).run(capture_stdout=True, capture_stderr=True)
#     audio_data = np.frombuffer(out, dtype=np.int16)

#     # Вычисление duration
#     sample_rate = 44100
#     duration = len(audio_data) / sample_rate

#     # Подсчёт пиков
#     chunk_size = len(audio_data) // samples
#     peaks = []
#     for i in range(samples):
#         start = i * chunk_size
#         end = start + chunk_size
#         chunk = audio_data[start:end]
#         if len(chunk) > 0:
#             max_val = int(np.max(chunk))
#             min_val = int(np.min(chunk))

#             # Нормализация значений пиков
#             normalized_max = max_val / 32768.0  # Преобразуем в диапазон от -1 до 1
#             normalized_min = min_val / 32768.0
#             peaks.extend([normalized_max, normalized_min])

#     # Сохраняем JSON
#     with open(output_json, 'w') as f:
#         json.dump({
#             "sample_rate": sample_rate,
#             "peaks": peaks,
#             "bits": 16,
#             "duration": duration
#         }, f)

def generate_peaks_3(audio_file, output_json, max_samples=100000, peaks_per_second=20):

    # out, _ = ffmpeg.input(audio_file).output('pipe:1', format='s16le', ac=1, ar=44100).run(capture_stdout=True, capture_stderr=True)
    out, _ = ffmpeg.input(audio_file, loglevel='quiet').output('pipe:1', format='s16le', ac=1, ar=44100).run(capture_stdout=True)

    audio_data = np.frombuffer(out, dtype=np.int16)


    # Вычисление duration
    sample_rate = 44100
    duration = len(audio_data) / sample_rate


    # Авто-расчет количества чанков
    samples = int(duration * peaks_per_second)
    samples = min(samples, max_samples)  # Ограничение сверху


    # Подсчёт пиков с учётом всех данных
    chunks = np.array_split(audio_data, samples)

    peaks = []
    for chunk in chunks:
        if len(chunk) > 0:
            max_val = int(np.max(chunk))
            min_val = int(np.min(chunk))

            # Нормализация значений пиков
            normalized_max = max_val / 32768.0
            normalized_min = min_val / 32768.0
            peaks.extend([normalized_max, normalized_min])

    # Сохраняем JSON
    with open(output_json, 'w') as f:
        json.dump({
            "sample_rate": sample_rate,
            "peaks": peaks,
            "bits": 16,
            "duration": duration
        }, f)


In [ ]:
generate_peaks_3(audio_file, 'Aitmatov_turnalar.json')

In [ ]:
from google.colab import files
files.download('Aitmatov_turnalar.json')


### Previous

In [ ]:
def generate_peaks(audio_file, output_json, samples=1000):
    # Используем FFmpeg для извлечения аудиоданных
    out, _ = ffmpeg.input(audio_file).output('pipe:1', format='s16le', ac=1, ar=44100).run(capture_stdout=True, capture_stderr=True)
    audio_data = np.frombuffer(out, dtype=np.int16)

    # Усреднение для пиков
    chunk_size = len(audio_data) // samples
    peaks = []
    for i in range(samples):
        start = i * chunk_size
        end = start + chunk_size
        chunk = audio_data[start:end]
        if len(chunk) > 0:
            max_val = np.max(chunk)
            min_val = np.min(chunk)
            # Convert NumPy int16 to native Python int before appending to peaks
            peaks.append([int(max_val), int(min_val)])

    # Сохраняем в JSON
    with open(output_json, 'w') as f:
        json.dump({
            "sample_rate": 44100,
            "peaks": peaks,
            "bits": 16
        }, f)

In [ ]:
def generate_peaks_2(audio_file, output_json, samples=3000):
    # Получаем точную длительность
    probe = ffmpeg.probe(audio_file)
    duration = float(probe['streams'][0]['duration'])

    # Уменьшаем частоту дискретизации для длинных файлов
    target_sample_rate = 22050  # Вдвое меньше стандартной 44100

    out, _ = (
        ffmpeg.input(audio_file)
        .output('pipe:1', format='s16le', ac=1, ar=target_sample_rate)
        .run(capture_stdout=True, capture_stderr=True)
    )

    audio_data = np.frombuffer(out, dtype=np.int16)

    # Агрессивное усреднение для длинных файлов
    chunk_size = len(audio_data) // samples
    peaks = []

    for i in range(samples):
        start = i * chunk_size
        end = start + chunk_size
        chunk = audio_data[start:end]

        if len(chunk) > 0:
            # Берем 95-й перцентиль для сглаживания пиков
            max_val = np.percentile(chunk, 95)
            min_val = np.percentile(chunk, 5)
            peaks.append([float(max_val), float(min_val)])

    # Сохраняем с оптимизированной структурой
    with open(output_json, 'w') as f:
        json.dump({
            "sample_rate": target_sample_rate,
            "peaks": peaks,
            "bits": 16,
            "duration": duration
        }, f, separators=(',', ':'))  # Минимизируем размер JSON